In [1]:
ORIGIANL_DATA_FILE = './../data/project_brazil/sudeste.csv'
PROCESSED_DATA_FILE = './../data/project_brazil/processed.csv'

---

<p>Имеется набор данных почасовых наблюдений погодных условий от 122 погодных станций Юго-Восточного региона Бразилии. В регион входят штаты: Рио-де-Жанейро, Сан-Паулу, Минас-Жерайс и Эспириту-Санту. Данные собираются с 2000 года (однако не все станции начали работы по сбору именно с этого года). Источником данных является INMET (Национальный Метеорологический Институт - Бразилия). Описание и ссылку на скачивание данных можно найти на странице <a href="https://www.kaggle.com/PROPPG-PPG/hourly-weather-surface-brazil-southeast-region">Kaggle Datasets</a>.

**Описания признаков в датасете**<br><br>
<b>wsid</b> (Numeric) - Уникальный идентификатор погодной станции<br>
<b>wsnm</b> (String) - Наименование погодной станции<br>
<b>elvt</b> (Numeric) - Высота расположения погодной станции<br>
<b>lat</b> (Numeric) - Широта месторасположения погодной станции (град.)<br>
<b>lon</b> (Numericw) - Долгота месторасположения погодной станции (град.)<br>
<b>inme</b> (String) - INMET-код станции для области<br>
<b>city</b> (String) - Город<br>
<b>prov</b> (String) - Штат (провинция)<br>
<b>mdct</b> (DateTime) - Дата и время наблюдения<br>
<b>date</b> (DateTime) - Дата наблюдения<br>
<b>yr</b> (Numeric) - Год наблюдения (2000-2016)<br>
<b>mo</b> (Numeric) - Месяц наблюдения (1-12)<br>
<b>da</b> (Numeric) - День наблюдения (1-31)<br>
<b>hr</b> (Numeric) - Час наблюдения (0-23)<br>
<b>prcp</b> (Numeric) - Количество осадков за последний час (мм)<br>
<b>stp</b> (Numeric) - Давление воздуха (мгновенное) (гПа)<br>
<b>smax</b> (Numeric) - Максимальное давление воздуха за последний час (гПа)<br>
<b>smin</b> (Numeric) - Минимальное давление воздуха за последний час (гПа)<br>
<b>gbrd</b> (String) - Солнечное излучение (кДж/м2)<br>
<b>temp</b> (Numeric) - Температура воздуха (мгновенная) (град. Цельсия)<br>
<b>dewp</b> (Numeric) - Температура точки росы (мгновенная) (град. Цельсия)<br>
<b>tmax</b> (Numeric) - Максимальная температура за последний час (град. Цельсия)<br>
<b>dmax</b> (Numericw) - Максимальная температура точки росы за последний час (град. Цельсия)<br>
<b>tmin</b> (Numeric) - Минимальная температура за последний час (град. Цельсия)<br>
<b>dmin</b> (Numeric) - Минимальная температура точки росыза последний час (град. Цельсия)<br>
<b>hmdy</b> (Numeric) - Относительная влажность (мгновенная) (%)<br>
<b>hmax</b> (Numeric) - Максимальная относительная влажность за последний час (%)<br>
<b>hmin</b> (Numeric) - Минимальная относительная влажность за последний час (%)<br>
<b>wdsp</b> (String) - Скорость ветра (м/с)<br>
<b>wdct</b> (Numeric) - Направление ветра (град) (0-360)<br>
<b>gust</b> (String) - Порывы ветра (м/с)<br>

Данный датасет может быть использован для опробования моделей машинного обучения в области прогнозирования погоды. Явно выделенной целевой переменной нет, а на Kaggle предлагается прогнозировать количество осадков или температуру. Мы попробуем спрогнозировать осадки, т.е. целевой меткой будет являться признак **prcp**. Подробнее последует далее.
<p>Необходимо учесть тот факт, что данные по осадкам хранятся для того часа, для которого актуальны и прочие наблюдения параметров. Для прогнозирования осадков на час, следующий за наблюдаемыми данными, в тренировочном датасете надо выполнить сдвиг значений целевой переменной на один шаг "вперед".
<p>Учитывая, что размер датасета значительный (1.72 Гб), сделаем предобработку файла данных, чтобы не загружать весь датасет в память. В простом предварительном взгляде на данные было выявлено, что примерно в 86% исходных данных отсутствует значение признака **prcp**. Выполним фильтрацию таких данных, оставив для дальнейшего анализа и обработки только корректные.
<p>Также стоит отметить, что, несмотря на присутствие в данных признаков с датой и временем, датасет не стоит расматривать как упорядоченный (какого-то тренда в погоде с течением времени не может быть в такой короткий временной отрезок, как 10-20 лет). Соответственно, эти признаки несут категориальный характер.

In [2]:
%%time

with open(ORIGIANL_DATA_FILE, 'r', encoding='utf-8') as in_file, open(PROCESSED_DATA_FILE, 'w', encoding='utf-8') as out_file:

    total_lines = 0
    processed_lines = 0

    print('Обработано строк:', end=' ', flush=True)

    # read full set
    for line in in_file:

        total_lines += 1

        # write dataset header line
        if total_lines == 1:
            out_file.write(line)
            continue

        # progress indication
        if total_lines % 1000000 == 0:
            print('{}M'.format(total_lines // 1000000), end=' ', flush=True)

        # serch 'prcp' field
        prcp_field = line.split(',')[14]

        if not prcp_field:
            continue
            
        # write line
        processed_lines += 1
        
        out_file.write(line)

    print()
    print('Всего обработано строк (с заголовком): {}'.format(total_lines))
    print('Отфильтрованные строки: {}'.format(processed_lines))

Обработано строк: 1M 2M 3M 4M 5M 6M 7M 8M 9M 
Всего обработано строк (с заголовком): 9779169
Отфильтрованные строки: 1407984
Wall time: 28.8 s
